## Lab 5: Q-Vectors
In this week's lab, we will be creating Q-Vector analyses to supplement Lab 5.  

<br />

### Useful Documentation
1. Xarray open_dataset:  https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html
2. Matplotlib Contour: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html
3. Matplotlib Quiver: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html
4. Cartopy Feature: https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html
5. MetPy Units: https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html
6. Datetime: https://docs.python.org/3/library/datetime.html
7. MetPy q_vector: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.q_vector.html
8. MetPy Divergence: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.divergence.html

<br />

### Tutorial
This week's, we wish to plot Q-Vectors and Q-Vector divergence using MetPy's q_vector and divergence functions.  This week's lab begins with a short tutorial on how to calculate and plot both the Q-Vectors and the Q-Vector divergence.

<br />

1. You are given some sample data below.  To calculate Q-Vectors, we need temperature, the u- and v-wind components, pressure, latitude, and longitude. Note: after initially defining these data as separate numpy arrays with MetPy units, the data are saved into an xarray Dataset to mimic the structure of gridded atmospheric datasets.

In [ ]:
import numpy as np
from metpy.units import units
from scipy.ndimage import gaussian_filter
import xarray as xr

# Define atmospheric variables on an idealized 5x5 grid.
temperature = np.array([[273, 273, 273, 273],
                        [274, 274, 274, 274],
                        [276, 276, 276, 276],
                        [278, 278, 278, 278],
                        [279, 279, 279, 279]])

pressure = np.array([[850, 850, 850, 850],
                     [850, 850, 850, 850],
                     [850, 850, 850, 850],
                     [850, 850, 850, 850],
                     [850, 850, 850, 850]])

u_wind = np.array([[0, 2, 5, 10],
                   [0, 2, 5, 10],
                   [0, 2, 5, 10],
                   [0, 2, 5, 10],
                   [0, 2, 5, 10]])

v_wind = np.array([[-10, -10, -10, -10],
                   [-2, -2, -2, -2],
                   [0, 0, 0, 0],
                   [2, 2, 2, 2],
                   [10, 10, 10, 10]])

latitude = np.array([41.5,
                     41.25,
                     41,
                     40.75,
                     40.5])

longitude = np.array([-100,-99.75,-99.5,-99.25])

# Combine these data arrays into an xarray Dataset.
ds = xr.Dataset(
    data_vars=dict(
        temperature=(["latitude", "longitude"], temperature),
        u=(["latitude", "longitude"], u_wind),    
        v=(["latitude", "longitude"], v_wind),        
        pressure=(["latitude", "longitude"], pressure)
    ),
    coords=dict(
        longitude=(["longitude"], longitude),
        latitude=(["latitude"], latitude),
    ),
    attrs=dict(description="Idealized Data"),
)

# Assign units
ds.u.attrs = {"units" : "mps"}
ds.v.attrs = {"units" : "mps"}
ds.temperature.attrs = {"units" : "kelvin"}
ds.pressure.attrs = {"units" : "hPa"}
ds.latitude.attrs = {"units" : "degrees_north"}
ds.longitude.attrs = {"units" : "degrees_east"}

ds

<br />

<br />

2. Raw Q-Vector data can be messy, with substantial mesoscale variability. To emphasize synoptic-scale features, however, we want to smooth the data. Since the vectors' angles are important, however, we need to smooth our data before we calculate the Q-Vectors.  The code below applies a Gaussian filer to the data, then saves the smoothed data back to the xarray.  A small sigma value (0.5) is used for these idealized data, such that little smoothing is applied, but a larger sigma value (such as 3) is more appropriate for real-world datasets..



In [ ]:
ds.temperature.values = gaussian_filter(ds.temperature.values,0.5)
ds.u.values = gaussian_filter(ds.u.values,0.5)
ds.v.values = gaussian_filter(ds.v.values,0.5)

<br /><br />

3. We are now ready to calculate the Q-Vectors.  Below, the MetPy Q-Vector function is used to calculate the u and v components of the Q-Vector using the temperature, pressure, u- and v-wind component data.


In [ ]:
import metpy.calc as calc
q_u, q_v = calc.q_vector(ds.u, ds.v, ds.temperature, ds.pressure)
q_v

<br /><br />
4. Next, we plot the Q-Vectors using matplotlib's quiver function.  The quiver function takes the following arguments in the order listed:
- Longitude
- Latitude
- Vector u component
- Vector v component
- The data's coordinate system.  For a latitude/longitude grid on a regular map projection, this is typically PlateCarree.
- Scale.  The number provided for this argument means that a vector with that magnitude will be drawn at the default length, with other vector magnitudes drawn proportional to this one. Thus, smaller values result in longer vectors and larger values result in smaller vectors.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
%matplotlib inline

# Create a small figure.
fig = plt.figure(figsize=(2,2), dpi=300)

# Use a Lambert conformal projection for plotting.
ax = plt.subplot(projection=crs.LambertConformal(central_longitude=-99.5, central_latitude=41.0,standard_parallels=(41.25,41.75)))

# Create spacing between vectors to make them legible.
# This option skips every other data point.
spacing = slice(None, None, 1)

# Plot the vectors.
ax.quiver(ds.longitude.values[spacing],
          ds.latitude.values[spacing],
          q_u.values[spacing, spacing],
          q_v.values[spacing, spacing],
          transform=crs.PlateCarree(),
          scale=2 * 10**-10)

# Add a descriptive title.
plt.title("Q-Vectors", weight="bold", size=5)


<br /><br />

5. We are typically interested in where Q-Vectors are converging (indicating forcing for ascent) and where the vectors are diverging (indicating forcing for descent).  We can obtain the Q-Vector divergence by using MetPy's divergence function:

In [ ]:
divergence = calc.divergence(q_u, q_v)
divergence

<br /><br />

6. Finally, we can plot the Q-Vectors and their divergence on the same map.  Note: the Q-Vector divergence is multiplied by 10<sup>17</sup> so that the contour values are easier to interpret, and the color table is reversed so that red shading indicates ascent.

In [ ]:
# Create the figure.
fig = plt.figure(figsize=(2,2), dpi=300)

# Use a Lambert conformal projection for plotting.
ax = plt.subplot(projection=crs.LambertConformal(central_longitude=-99.5, central_latitude=41.0,standard_parallels=(41.25,41.75)))

# Contour the Q-Vector divergence.
cont = ax.contourf(ds.longitude.values, ds.latitude.values, divergence.values* 10**17, np.arange(-20,22.5,2.5), cmap="bwr_r", transform=crs.PlateCarree(), extend="both", alpha=0.75)

# Create a divergence colorbar, then add a label to it and adjust the tick labels
# to be smaller (for this small plot) so they are legible.
cb = plt.colorbar(cont)
cb.set_label("Q-Vector Divergence $m kg^{-1} s^{-1}$ x $10^{-17}$", size=4)
cb.ax.tick_params(labelsize=3)

# Create spacing between vectors to make them legible.
# This option skips every other data point.
spacing = slice(None, None, 1)

# Plot the vectors.
ax.quiver(ds.longitude.values[spacing],
          ds.latitude.values[spacing],
          q_u.values[spacing, spacing],
          q_v.values[spacing, spacing],
          transform=crs.PlateCarree(),
          scale=2 * 10**-10)

# Add a descriptive title.
plt.title("Q-Vectors and Q-Vector Divergence ($m kg^{-1} s^{-1}$ x $10^{-17}$)", weight="bold", size=4)


<br /><br />

### Part II Instructions

The 1800 UTC 22 February 2023 GFS analysis has been locally downloaded for you to use with this lab.  The location of these data is provided for you below, as is the filename convention if you wish to use a datetime object to select your time of interest.

Using the downloaded GFS data, create the following maps:

<ul>
    <li>500 hPa Geopotential Height and Absolute Vorticity
    <li>925 hPa Geopotential Height and Absolute Vorticity
    <li>700 hPa Geopotential Height, Temperature, Q-Vectors, and Q-Vector Divergence
</ul>

Be sure your maps follow the "good map" guidelines, and don't forget to import the necessary packages before you start coding. Make sure that your plots incorporate the points included in the second code block below.

In [ ]:
from datetime import datetime

# Variable to specify datetime.
time = 

# Data location.
data_location = "/data/AtmSci360/Synp2/Lab_5/"

# Filename convention.
file_name = f"{time:%m%d%y_%H}_gfs.grib2"

In [ ]:
ax.text(-91, 39.25, "A", size = 16, weight="bold", transform=crs.PlateCarree())
ax.text(-96, 47.5, "B", size = 16, weight="bold", transform=crs.PlateCarree())

<br /><br />

### Part IV Instructions

So you can complete this part of the lab, GFS analyses since 1200 UTC 15 March 2023 have been locally downloaded for you to use to complete this part of the lab.  The location of these data is provided for you below, as is the filename convention if you wish to use a datetime object to select your time of interest.

Using the downloaded GFS data, create the following maps:

<ul>
    <li>500 hPa Geopotential Height and Absolute Vorticity
    <li>925 hPa Geopotential Height and Absolute Vorticity
    <li>700 hPa Geopotential Height, Temperature, Q-Vectors, and Q-Vector Divergence
</ul>

Be sure your maps follow the "good map" guidelines, and don't forget to import the necessary packages before you start coding.

In [ ]:
# Variable to specify datetime.
time = 

# Data location.
data_location = "/data/AtmSci360/Synp2/Lab_5/"

# Filename convention.
file_name = f"{time:%m%d%y_%H}_gfs.grib2"